# Evaluacion y analisis de modelos

In [ ]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

reg_dir = Path("../02_lineal_regression_results/regression_lineal")
prep_dir = Path("../01_preprocessing_results/preprocessing")

objetivo = "Total_Amount"
train = pd.read_csv(prep_dir / "T_train_final_objetivo.csv")
test = pd.read_csv(prep_dir / "T_test_final_objetivo.csv")

X_train = train.drop(columns=[objetivo])
y_train = train[objetivo]
X_test = test.drop(columns=[objetivo])
y_test = test[objetivo]

lin = joblib.load(reg_dir / "modelo_reg_lineal.pkl")
rf = joblib.load(reg_dir / "modelo_random_forest.pkl")


In [ ]:
# Evaluar los modelos cargados

def evaluar(nombre, modelo):
    preds = modelo.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    rmse = float(np.sqrt(mse))
    mae = float(mean_absolute_error(y_test, preds))
    r2 = float(r2_score(y_test, preds))
    return {"modelo": nombre, "mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "preds": preds}

res_lin = evaluar("LinearRegression", lin)
res_rf = evaluar("RandomForestRegressor", rf)

resultados = pd.DataFrame([
    {k: v for k, v in res_lin.items() if k != "preds"},
    {k: v for k, v in res_rf.items() if k != "preds"},
])
resultados


In [ ]:
# Importancias de variables para el RandomForest (modelo ganador)
importancias = pd.Series(rf.feature_importances_, index=X_train.columns)
importancias_10 = importancias.sort_values(ascending=False).head(10)
importancias_df = importancias.sort_values(ascending=False).reset_index()
importancias_df.columns = ["feature", "importance"]
importancias_df.to_csv(reg_dir / "importancias_random_forest.csv", index=False)
importancias_10


In [ ]:
# Resumen y guardado
resultados.to_csv(reg_dir / "resumen_evaluacion.csv", index=False)
mejor = max([res_lin, res_rf], key=lambda r: r["r2"])
print("Mejor modelo:", mejor["modelo"], "R2=", mejor["r2"])
